## Spark Stream Practice (Solutions)


In [ ]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

### Spark Stream with socket

In another tab open a terminal and open a socket using 
    
    nc -l -p 9090

This will allow you to open a TCP socket and to manually send data

Connect Spark to the stream

In [ ]:
lines = (spark.readStream                        
.format("socket")                                 
.option("host","localhost")
.option("port","9090")
.load())   

For now, let's just inspect the stream

In [ ]:
lines.writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

Now let's do a word count

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

 # Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

### Spark streaming with a Kafka

We will use the other notebook "Kafka-producer" to handle Kafka

In [ ]:
kafka_server = "kafka2:9093"   

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "words")                       # Subscribe to the "en" Kafka topic - edits of English wikipedia pages
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()                                          # Load the DataFrame
)

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

 # Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

query.awaitTermination()

### Spark Structured Stream

Let's start with an easy example that reads from file

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

schema = StructType(
      [
        StructField("RecordNumber", IntegerType(), True),
        StructField("Zipcode", StringType(), True),
        StructField("ZipCodeType", StringType(), True),
        StructField("City", StringType(), True),
        StructField("State", StringType(), True),
        StructField("LocationType", StringType(), True),
        StructField("Lat", StringType(), True),
        StructField("Long", StringType(), True),
        StructField("Xaxis", StringType(), True),
        StructField("Yaxis", StringType(), True),
        StructField("Zaxis", StringType(), True),
        StructField("WorldRegion", StringType(), True),
        StructField("Country", StringType(), True),
        StructField("LocationText", StringType(), True),
        StructField("Location", StringType(), True),
        StructField("Decommisioned", StringType(), True)
      ]
    )

In [ ]:
zips = spark.readStream.schema(schema).json('data/zipcode')

In [ ]:
zips.printSchema()

We can look at the stream

In [ ]:
zips.writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

And we can compute some aggregation on it

In [ ]:
groupDF = zips.select("Zipcode").groupBy("Zipcode").count()

groupDF.printSchema()

groupDF.writeStream \
.format("console") \
.outputMode("complete") \
.start() \
.awaitTermination()

In [ ]:
groupDF = zips.select("LocationType").groupBy("LocationType").count()

groupDF.printSchema()

groupDF.writeStream \
.format("console") \
.outputMode("complete") \
.start() 

### Window operator

In [ ]:
from pyspark.sql.functions import window

kafka_server = "kafka1:9092"   

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "words")                       # Subscribe to the "en" Kafka topic - edits of English wikipedia pages
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()                                          # Load the DataFrame
)

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds")).count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

#### Sliding time window

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds", "2 seconds")).count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

#### Other aggregations

In [ ]:
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds")).count()

wordCounts = wordCounts.orderBy(col("count").desc())

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

In [ ]:
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds")).count()

wordCounts = wordCounts.max_by(col("count"))

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

### Wikipedia edits

https://stream.wikimedia.org/v2/ui/#/?streams=mediawiki.recentchange

The kafka producer in the other notebook creates 1 stream for each event type (edits, categorize, etc.. but not for the log event)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType

schema = StructType(
      [
        StructField("domain", StringType(), True),
        StructField("namespace", StringType(), True),
        StructField("uri", StringType(), True),
        StructField("title", StringType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("user_name", StringType(), True),
        StructField("user_type", StringType(), True),
        StructField("old_length", StringType(), True),
        StructField("new_length", StringType(), True),
      ]
    )

In [ ]:
from pyspark.sql.functions import window
from pyspark.sql.functions import from_json
kafka_server = "kafka1:9092"   

wiki_df = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "edit")                       # Subscribe to the "en" Kafka topi
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
)

In the last mimute how many edits are done by bots and by humans

In [ ]:
data = wiki_df.select("parsed_value.*")

group_df = data.groupBy(col("user_type"), window(col("timestamp"),"1 minutes")).count()

query = group_df \
          .writeStream \
          .outputMode("update") \
          .format("console") \
          .start()

query.awaitTermination()

Select the 10 most edited pages

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data.groupBy(col("uri"), window(col("timestamp"),"2 minutes")).count().orderBy(col("count").desc()).limit(10)

query = group_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start() \
         

query.awaitTermination()

Who are the top editors of the english wikipedia?

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data \
.filter(col("domain") == "en.wikipedia.org") \
.groupBy("user_name").count() \
.orderBy(col("count").desc())

query = group_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start() \
         

query.awaitTermination()

Are they humans or bot?

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data \
.filter(col("domain") == "en.wikipedia.org") \
.groupBy("user_name","user_type").count() \
.orderBy(col("count").desc())

query = group_df \
.writeStream \
.outputMode("complete") \
.format("console") \
.start() \


query.awaitTermination()

How many edits added something to the page?

In [ ]:
from pyspark.sql.functions import col, row_number, when, lit

df = wiki_df.select("parsed_value.*")

group_df = df \
.withColumn("op_type", when((df.new_length - df.old_length > 0),lit("ADD")).otherwise(lit("REMOVE"))) \
.groupBy(col("op_type")).count()

query = group_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start() \
         

query.awaitTermination()

#### Joining streams

Let's register to the "new" stream

In [5]:
from pyspark.sql.functions import window
from pyspark.sql.functions import from_json
kafka_server = "kafka1:9092"   

new_df = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "new")                       # Subscribe to the "en" Kafka topi
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
)

Let's see if there are users who add and edit things right after

In [6]:
from pyspark.sql.functions import expr

df_edit = wiki_df.select("parsed_value.*").select("user_name","user_type","uri","title","timestamp") \
.withColumnRenamed("timestamp","editTime").withColumnRenamed("user_name","editName")

df_new = new_df.select("parsed_value.*").select("user_name","user_type","uri","title","timestamp") \
.withColumnRenamed("timestamp","newTime").withColumnRenamed("user_name","newName")

#joined_streams_df = (df_edit
#            .join(df_new, "user_name"))

joined_streams_df = (df_edit
           .join(df_new, expr("""
    editName = newName AND
    newTime >= editTime AND
    newTime <= editTime + interval 5 minute
    """))).select(col("editName"))

query = joined_streams_df.writeStream.outputMode("append").format("console").start()

query.awaitTermination()

23/10/16 15:12:03 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bdfa487e-51f8-403d-8190-091936f7a371. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------+
|          editName|
+------------------+
| Materialscientist|
|       Schlosser67|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------+
|          editName|
+------------------+
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------+
|          editName|
+------------------+
|           Mannivu|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------+
|          editName|
+------------------+
|             Ghaly|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



...

-------------------------------------------
Batch: 28
-------------------------------------------
+----------+
|  editName|
+----------+
|NeverDoING|
|NeverDoING|
|NeverDoING|
|      Lupe|
|      Lupe|
|      Lupe|
|      Lupe|
|   Crowsus|
+----------+



-------------------------------------------
Batch: 29
-------------------------------------------
+------------------+
|          editName|
+------------------+
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



-------------------------------------------
Batch: 30
-------------------------------------------
+------------------+
|          editName|
+------------------+
|     Рейму Хакурей|
|     Рейму Хакурей|
| Materialscientist|
| Materialscientist|
| Materialscientist|
| Materialscientist|
| Materialscientist|
| Materialscientist|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|   YuitoAkatsukiJP|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows



KeyboardInterrupt: 

-------------------------------------------
Batch: 31
-------------------------------------------
+------------------+
|          editName|
+------------------+
|        NeverDoING|
|        NeverDoING|
|        NeverDoING|
|        Sangjinhwa|
| Materialscientist|
| Materialscientist|
| Materialscientist|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
|InternetArchiveBot|
+------------------+
only showing top 20 rows

